In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Adrenocortical_Cancer/GSE67766'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Cancer Cells Hijack PRC2 to Modify Multiple Cytokine Pathways"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: SW-13']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Based on the sample characteristics dictionary provided in the output of STEP 1, 
# it appears there are no explicit rows for "Adrenocortical_Cancer", "age", or "gender".
# Additionally, the data description indicates the use of cell lines; thus these human traits are not applicable.

# Assuming gene expression data might be available from the GEO database series is_gene_available will be set to True.
is_gene_available = True

def convert_trait(value):
    try:
        raw_value = value.split(':')[-1].strip().lower()
        if raw_value in ["yes", "positive", "present"]:
            return 1
        elif raw_value in ["no", "negative", "absent"]:
            return 0
    except:
        pass
    return None

def convert_age(value):
    try:
        raw_value = value.split(':')[-1].strip()
        if raw_value.isdigit():
            return int(raw_value)
    except:
        pass
    return None

def convert_gender(value):
    try:
        raw_value = value.split(':')[-1].strip().lower()
        if raw_value in ["female", "f"]:
            return 0
        elif raw_value in ["male", "m"]:
            return 1
    except:
        pass
    return None

save_cohort_info('GSE67766', './preprocessed/Adrenocortical_Cancer/cohort_info.json', is_gene_available, trait_row is not None)
